In [58]:
import os
from typing import Final

from bayes_opt import BayesianOptimization
from bayes_opt.event import Events
from bayes_opt.logger import JSONLogger
from bayes_opt.util import load_logs
from numpy import where
from pandas import DataFrame, Series, concat, read_csv
from sklearn.metrics import f1_score
from sklearn.svm import OneClassSVM

NUM_TRIALS: Final[int] = 5
LOGS_PATH: Final[str] = "../reports/logs_sequence.log"
logger = JSONLogger(path=LOGS_PATH, reset=False)

In [59]:
X_train = read_csv("../data/PAMAP2/x_train_data.csv")
X_test = read_csv("../data/PAMAP2/x_test_data.csv")
y_train = read_csv("../data/PAMAP2/y_train_data.csv")
y_test = read_csv("../data/PAMAP2/y_test_data.csv")

X_train["activity"] = y_train  # First 80% of the data
X_test["activity"] = y_test  # Last 20% of the data

# MIN_SAMPLES = X_train["activity"].value_counts().min()
MIN_SAMPLES = X_train["activity"].value_counts().sort_values().iloc[3]
MAXIMAZED = False

In [60]:
def objective_function(nu: float, gamma: float) -> float:
    """
    Objective function to optimize F1-Score on the train and test set.

    Args:
        nu (float): nu param to evaluate.
        gamma (float): gamma param to evaluate.

    Returns:
        float: F1-Score on the test set of this iteration.
    """
    oc_svm = OneClassSVM(kernel="rbf", nu=nu, gamma=gamma).fit(training_data)

    f1_train = f1_score(
        train_targets,
        where(oc_svm.predict(training_data) == 1, False, True),
        average="macro",
    )
    f1_test = f1_score(
        test_targets,
        where(oc_svm.predict(testing_data) == 1, False, True),
        average="macro",
    )
    print(f"\nF1 Score (Train): {f1_train}\nF1 Score (Test): {f1_test}")
    return float(f1_test)

In [61]:
models: dict[int, dict] = {}
training_data: DataFrame
testing_data: DataFrame
train_targets: Series
test_targets: Series

optimizer = BayesianOptimization(
    objective_function, {"nu": (0.01, 0.5), "gamma": (1e-4, 1e-1)}, random_state=42
)
if not os.path.exists(LOGS_PATH):
    with open(LOGS_PATH, "w") as fp:
        pass
optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)

In [62]:
for i in range(1, len((activities := X_train["activity"].unique()))):
    training = (  # picks the first n samples of each class
        X_train[X_train["activity"].isin(activities[:i])]
        .groupby("activity")
        .head(MIN_SAMPLES)
        .copy()
    )
    testing = X_test[X_test["activity"] == activities[i]].head(MIN_SAMPLES)
    training.loc[:, "isNovelty"], testing.loc[:, "isNovelty"] = False, True
    novelty = concat(
        [testing, training.sample(n=int(0.2 * len(training)), random_state=42)]
    )
    training_data = training.drop(columns=["isNovelty"])
    train_targets = training["isNovelty"]
    # only current activity (as novelty)
    testing_data = novelty.drop(columns=["isNovelty"])
    test_targets = novelty["isNovelty"]

    load_logs(optimizer, logs=[LOGS_PATH])
    print("New optimizer is now aware of {} points.".format(len(optimizer.space)))

    if not MAXIMAZED:
        optimizer.maximize(init_points=5, n_iter=25)
        MAXIMAZED = True

    else:
        load_logs(optimizer, logs=[LOGS_PATH])
        print(f"Already maximized, sugesting new {NUM_TRIALS} points")
        for i in range(NUM_TRIALS):
            MAX = optimizer.max["target"]
            print(
                "Next point:",
                next_point_to_probe := optimizer.suggest(),
                "Found the target:",
                target := objective_function(**next_point_to_probe),
            )
            optimizer.register(params=next_point_to_probe, target=target)
            if target > MAX:
                print("New best points found, continuing optimization")
                i = 0

    models[i] = optimizer.max  # type: ignore

New optimizer is now aware of 55 points.

F1 Score (Train): 0.48314606741573035
F1 Score (Test): 0.971047526089349

F1 Score (Train): 0.4121607694950189
F1 Score (Test): 0.87254290636244

F1 Score (Train): 0.4752851711026616
F1 Score (Test): 0.961128649128649

F1 Score (Train): 0.4880631843475139
F1 Score (Test): 0.9815047645139792

F1 Score (Train): 0.4628664699604495
F1 Score (Test): 0.945615908300795

F1 Score (Train): 0.4549276931897815
F1 Score (Test): 0.9297271541041512

F1 Score (Train): 0.4674136321195145
F1 Score (Test): 0.951770158713132

F1 Score (Train): 0.41074380165289254
F1 Score (Test): 0.8687794901065449

F1 Score (Train): 0.48044692737430167
F1 Score (Test): 0.9694424977412062

F1 Score (Train): 0.46909903201787045
F1 Score (Test): 0.9520057214717721

F1 Score (Train): 0.4674136321195145
F1 Score (Test): 0.9508270716238427

F1 Score (Train): 0.46900018618506795
F1 Score (Test): 0.9548259997553341

F1 Score (Train): 0.4734769230769231
F1 Score (Test): 0.958568001751097